In [10]:
from web3 import Web3
import pandas as pd
import requests
from dotenv import load_dotenv
import os

# Loading environment variables from .env file

load_dotenv()
GATEWAY = os.getenv("GATEWAY_URL")

In [11]:
w3 = Web3(Web3.HTTPProvider(GATEWAY))
print(f'Is the connection successful?: {w3.is_connected()}') # Should return True

Is the connection successful?: True


In [14]:
import asyncio
from web3 import Web3
from collections import defaultdict, deque
import statistics
import time

class EthereumAnomalyTracker:
    def __init__(self, infura_http_url: str):
        self.w3 = Web3(Web3.HTTPProvider(GATEWAY))
        if not self.w3.is_connected():
            raise ConnectionError("Could not connect to Infura HTTP")

        # Data buffers
        self.blocks = deque(maxlen=100)
        self.transactions = deque(maxlen=5000)
        self.events = deque(maxlen=5000)

        # Tracking activity per wallet
        self.wallet_activity = defaultdict(lambda: deque(maxlen=50))

    # -------- Fetch & Classify -------- #
    async def stream_new_blocks(self):
        """Listen for new blocks and classify their contents"""
        while True:
            block = self.w3.eth.get_block("latest", full_transactions=True)

            block_info = {
                "number": block.number,
                "timestamp": block.timestamp,
                "miner": block.miner,
                "gasUsed": block.gasUsed,
            }
            self.blocks.append(block_info)

            # Process each transaction
            for tx in block.transactions:
                tx_info = {
                    "hash": tx.hash.hex(),
                    "from": tx["from"],
                    "to": tx["to"],
                    "value": self.w3.from_wei(tx["value"], "ether"),
                    "gasPrice": tx["gasPrice"],
                }
                self.transactions.append(tx_info)

                # Track wallet activity
                self.wallet_activity[tx["from"]].append(time.time())

                # Detect anomalies
                self.detect_transaction_anomalies(tx_info)

            # Detect block anomalies
            self.detect_block_anomalies(block_info)

            await asyncio.sleep(12)  # ~avg block time

    # -------- Anomaly Detection -------- #
    def detect_block_anomalies(self, block):
        """Check block-level anomalies (e.g., gas spikes)"""
        gas_used = block["gasUsed"]
        if len(self.blocks) > 10:
            gas_values = [b["gasUsed"] for b in self.blocks]
            mean, stdev = statistics.mean(gas_values), statistics.pstdev(gas_values)
            if gas_used > mean + 3 * stdev:
                print(f"[ANOMALY] Block {block['number']} unusually high gas usage: {gas_used}")

    def detect_transaction_anomalies(self, tx):
        """Check tx-level anomalies"""
        # Large transfers
        if tx["value"] > 1000:  # >1000 ETH
            print(f"[ANOMALY] Large transfer detected: {tx['value']} ETH from {tx['from']}")

        # Gas price spikes
        if len(self.transactions) > 100:
            gas_prices = [t["gasPrice"] for t in self.transactions]
            mean, stdev = statistics.mean(gas_prices), statistics.pstdev(gas_prices)
            if tx["gasPrice"] > mean + 3 * stdev:
                print(f"[ANOMALY] Gas spike in tx {tx['hash']} : {tx['gasPrice']}")

        # Wallet activity (burst detection)
        recent_txs = self.wallet_activity[tx["from"]]
        if len(recent_txs) >= 5:  # more than 5 txs in 1 minute
            if recent_txs[-1] - recent_txs[0] < 60:
                print(f"[ANOMALY] Wallet {tx['from']} spamming transactions")

    # -------- Event Logs (ERC20 transfers / contract calls) -------- #
    def fetch_event_logs(self, contract_address, abi, from_block="latest"):
        """Fetch contract events (e.g., token transfers)"""
        contract = self.w3.eth.contract(address=contract_address, abi=abi)
        events = contract.events.Transfer.createFilter(fromBlock=from_block).get_all_entries()
        for e in events:
            self.events.append({
                "from": e["args"]["from"],
                "to": e["args"]["to"],
                "value": e["args"]["value"]
            })
            # Detect abnormal transfers
            if e["args"]["value"] > 1e24:  # large token transfer
                print(f"[ANOMALY] Large token transfer {e['args']['value']} from {e['args']['from']}")

    # -------- Runner -------- #
    async def run(self):
        await self.stream_new_blocks()


# ---------- Example Usage ----------
if __name__ == "__main__":
    tracker = EthereumAnomalyTracker(GATEWAY)
    asyncio.run(tracker.run())

RuntimeError: asyncio.run() cannot be called from a running event loop